## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-23-16-53-32 +0000,nyt,"For Greenland Tourism, Trump’s Interest Create...",https://www.nytimes.com/2026/01/23/travel/gree...
1,2026-01-23-16-52-00 +0000,wsj,Canada Retail Sales Pull Back After Rise in No...,https://www.wsj.com/business/retail/canada-ret...
2,2026-01-23-16-51-52 +0000,wapo,What Don Lemon did at the St. Paul church prot...,https://www.washingtonpost.com/investigations/...
3,2026-01-23-16-51-24 +0000,nyt,Hundreds of Minnesota Businesses on Strike in ...,https://www.nytimes.com/2026/01/23/us/minnesot...
4,2026-01-23-16-50-56 +0000,nyt,How a Major Winter Storm This Weekend Could Af...,https://www.nytimes.com/2026/01/22/travel/wint...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2480/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,77
160,new,22
26,ice,19
0,greenland,18
22,minnesota,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
73,2026-01-23-14-08-01 +0000,wapo,Trump withdraws Carney’s invitation to Board o...,https://www.washingtonpost.com/world/2026/01/2...,139
206,2026-01-23-02-31-45 +0000,cbc,Trump says he's withdrawing invitation for Car...,https://www.cbc.ca/news/politics/trump-board-o...,136
305,2026-01-22-21-19-43 +0000,bbc,"Board of Peace, Zelensky and Musk - What happe...",https://www.bbc.com/news/videos/c78vvvn4qnlo?a...,133
187,2026-01-23-03-44-07 +0000,nypost,Trump withdraws Canadian PM Mark Carney’s Boar...,https://nypost.com/2026/01/22/us-news/trump-wi...,132
20,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...,132


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
73,139,2026-01-23-14-08-01 +0000,wapo,Trump withdraws Carney’s invitation to Board o...,https://www.washingtonpost.com/world/2026/01/2...
326,67,2026-01-22-19-43-16 +0000,nyt,Judge Rejects DOJ’s Criminal Complaint Against...,https://www.nytimes.com/2026/01/22/us/politics...
162,59,2026-01-23-08-08-54 +0000,bbc,Is the US-TikTok deal a new reality for China'...,https://www.bbc.com/news/articles/c6200mz054yo...
203,51,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...
55,48,2026-01-23-15-04-40 +0000,wapo,"On Greenland, Europe stood up, Trump blinked a...",https://www.washingtonpost.com/world/2026/01/2...
194,44,2026-01-23-03-24-01 +0000,nypost,Arizona AG Kris Mayes wildly suggests resident...,https://nypost.com/2026/01/22/us-news/arizona-...
8,38,2026-01-23-16-45-06 +0000,nyt,"Ryan Wedding, Canadian Ex-Snowboarder Accused ...",https://www.nytimes.com/2026/01/23/world/canad...
286,37,2026-01-22-22-02-00 +0000,wsj,JPMorgan CEO Jamie Dimon Reaps $43 Million in ...,https://www.wsj.com/finance/banking/jpmorgan-c...
198,36,2026-01-23-03-00-00 +0000,wsj,When asked what connects his two spheres of in...,https://www.wsj.com/business/media/dana-whites...
307,33,2026-01-22-21-15-36 +0000,nypost,Rep. Chip Roy tears into Jack Smith over Arcti...,https://nypost.com/2026/01/22/us-news/texas-re...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
